In [5]:
import os
os.chdir( "/home/lucasfuzato/CODING/WINE" )

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from itertools import product

from src.aux_fun import *

tests_df = pd.DataFrame( columns = [ "n_neigh" , "weight" ] )

num_neigh = [ 25 , 50 , 100 , 200 , 250 ]
using = [ "uniform" , "distance" ]
for i , ( n_neigh , smote ) in enumerate( product( num_neigh , using ) ):
    tests_df.loc[ i ] = pd.Series( [ n_neigh , smote ] , [ "n_neigh" , "weight" ] )

print( tests_df )

  n_neigh    weight
0      25   uniform
1      25  distance
2      50   uniform
3      50  distance
4     100   uniform
5     100  distance
6     200   uniform
7     200  distance
8     250   uniform
9     250  distance


In [6]:
from sklearn.neighbors import KNeighborsClassifier as knn
seed_val = 47
X , y , X_2d , columns = class_get_data()

scores = [ "f1_score" , "precision" , "recall" ]
metrics = [ "mean" , "std" ]
data = { tup:list() for tup in product( scores , metrics ) }

for simu_id , ( n_neigh , weight ) in enumerate( product( num_neigh , using ) ):

    lst = []
    fold_iter = get_prepared_data( X , y , seed = seed_val , smote = False )
    for ( X_train , y_train ) , ( X_test , y_test ) in fold_iter:

        k_class = knn( n_neighbors = n_neigh , weights = weight )
        k_class.fit( X_train , y_train )

        ts_cls = k_class.predict( X_test )
        ts_f1 , prec , recall = bin_f1_score( ts_cls , y_test )

        lst.append( ( ts_f1 , prec , recall ) )
    
    arr = np.array( lst )
    means = arr.mean( axis = 0 )
    data[ ( "f1_score", "mean" ) ].append( means[ 0 ] )
    data[ ( "precision", "mean" ) ].append( means[ 1 ] )
    data[ ( "recall", "mean" ) ].append( means[ 2 ] )

    stds  = arr.std( axis = 0 )
    data[ ( "f1_score", "std" ) ].append( stds[ 0 ] )
    data[ ( "precision", "std" ) ].append( stds[ 1 ] )
    data[ ( "recall", "std" ) ].append( stds[ 2 ] )

print( pd.DataFrame( data ) )
print()

   f1_score           precision              recall          
       mean       std      mean       std      mean       std
0  0.572814  0.057950  0.632705  0.061526  0.393141  0.052587
1  0.729851  0.042825  0.774744  0.049614  0.595171  0.039268
2  0.571272  0.060649  0.651804  0.070158  0.329675  0.043281
3  0.741492  0.044511  0.798874  0.054798  0.569347  0.030835
4  0.546984  0.054622  0.645282  0.065519  0.252092  0.032303
5  0.765831  0.033066  0.839957  0.042185  0.543455  0.021633
6  0.537112  0.073897  0.655079  0.091961  0.183212  0.026227
7  0.775469  0.027931  0.868718  0.036795  0.495725  0.023021
8  0.537741  0.064219  0.658788  0.079145  0.174600  0.021618
9  0.775296  0.026311  0.872396  0.036663  0.483994  0.019890

